# 08 — Built-in Functions: `.bind()` and `.assign()`

This notebook introduces two powerful LCEL helpers used when composing chains:

- **`.bind()`** — freeze (attach) provider-specific arguments or generation parameters onto a runnable (usually an LLM) at build time.
- **`.assign()`** — add new keys to the **dictionary output** of a runnable (commonly a `RunnableParallel`) without redefining the whole structure.

In [1]:
# ╔══════════════════════════════════════════════════════╗
# ║ Setup: Load environment variables & initialize model ║
# ╚══════════════════════════════════════════════════════╝

import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq

chat_model = ChatOpenAI(model="gpt-4o-mini")
# chat_model = ChatGroq(model="llama-3.1-70b-versatile")

print("✅ Environment loaded and model ready.")

✅ Environment loaded and model ready.


## `.bind()` — freeze provider/config arguments on a runnable

`Runnable.bind(**kwargs)` lets you **inject extra arguments directly into the runnable** (usually a chat model) at chain-construction time. Think of it as *freezing* certain parameters so you don’t have to pass them on every call.

**Typical uses:**
- Provider-specific features (e.g., OpenAI **tools**, Anthropic **tool use**).
- Generation params like `temperature`, `max_tokens`, `stop`, etc.
- Low-level control when the high-level helpers are not enough.

### Example: bind generation parameters

We bind temperature and max tokens once, then reuse the model in a chain.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Answer briefly in no more than 2 sentences: {question}"
)

# Freeze provider params into the runnable
model_tuned = chat_model.bind(temperature=0, max_tokens=120)

chain = prompt | model_tuned | StrOutputParser()
print(chain.invoke({"question": "What is LCEL in LangChain?"}))

LCEL, or LangChain Event Log, is a feature in LangChain that allows users to track and log events during the execution of language model applications. It helps in monitoring, debugging, and analyzing the performance of the application by providing insights into the interactions and decisions made by the model.


### Example: tool calling (provider-specific via `.bind()`)

You can bind tool specifications directly (low-level) to an OpenAI model. The model may return **tool call instructions** in `additional_kwargs`.

In [ ]:
# --- Manual tool definition (JSON schema style) ---

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

chat_model = ChatOpenAI(model="gpt-4o-mini")
model_raw = chat_model.bind(tools=tools)

resp = model_raw.invoke("What's the weather in Madrid?")
print("Model output:\n", resp.content)
print("\nRaw tool_calls (if any):", getattr(resp, "tool_calls", None))

Model text:
 


> **Notes**
> - `.bind()` returns a **new runnable**; it does not mutate the original. You can reuse the base model.
> - You can chain multiple `.bind()` calls:
>   ```python
>   base = ChatOpenAI(model="gpt-4o-mini")
>   model = base.bind(temperature=0).bind(max_tokens=100)
>   ```

#### Using `.bind_tools()` (high-level helper)

You can define tools with the `@tool` decorator and bind them in a provider-agnostic way.

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

@tool
def get_current_weather(location: str, unit: str = "celsius") -> str:
    """Return dummy weather (example)."""
    return f"The weather in {location} is 20° {unit} (demo)."

# 2️⃣ Bind the tool to the model
model_with_tools = chat_model.bind_tools([get_current_weather])

# 3️⃣ Invoke with a user message
resp = model_with_tools.invoke("What's the weather in Madrid?")
print("Model output:\n", resp.content)

# 4️⃣ (Optional) Inspect if the model proposed a tool call
if hasattr(resp, "tool_calls") and resp.tool_calls:
    print("\nTool call proposed:\n", resp.tool_calls)

### Differences `.bind()` vs `.bind_tools()`

- **`.bind_tools()`** → high-level, recommended for tool calling. It abstracts provider specifics and uses a more standard interface.
- **`.bind()`** → low-level; you pass raw provider arguments (e.g., the exact `tools` schema that OpenAI expects). Useful when you need fine-grained control.

---
## `.assign()` — add new keys to a dict output

`Runnable.assign(**new_keys)` extends the **dictionary output** of a runnable with additional keys computed from the current output.

Most commonly you start from a `RunnableParallel` (which already returns a dict) and then **add** fields without redefining the whole mapping.

### Step 1: parallel block with only the original input

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = RunnableParallel({
    "original_input": RunnablePassthrough()
})

print(chain.invoke("whatever"))  # → {'original_input': 'whatever'}

### Step 2: add a new key with `.assign()`

In [ ]:
from langchain_core.runnables import RunnableLambda

def make_uppercase(data: dict) -> str:
    return data["original_input"].upper()

chain2 = (
    RunnableParallel({"original_input": RunnablePassthrough()})
    .assign(uppercase=RunnableLambda(make_uppercase))
)

print(chain2.invoke("whatever"))
# → {'original_input': 'whatever', 'uppercase': 'WHATEVER'}

### Step 3: add more fields progressively
You can call `.assign()` multiple times to keep enriching the output.

In [ ]:
chain3 = chain2.assign(
    length=RunnableLambda(lambda d: len(d["original_input"]))
).assign(
    has_space=RunnableLambda(lambda d: " " in d["original_input"]) 
)

print(chain3.invoke("LangChain rocks!"))
# → {'original_input': 'LangChain rocks!', 'uppercase': 'LANGCHAIN ROCKS!', 'length': 16, 'has_space': True}

### Example: prepare prompt variables then call a model

We first build a dict of variables, then pass it into a prompt → model → parser chain.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1) Build the variables dict using parallel + assign
vars_chain = (
    RunnableParallel({"question": RunnablePassthrough()})
    .assign(instruction=RunnableLambda(lambda d: "Answer briefly."))
)

# 2) Create prompt using those variables
prompt = ChatPromptTemplate.from_template(
    "{instruction}\nQuestion: {question}"
)

# 3) Compose: variables → prompt → model → parser
full_chain = vars_chain | prompt | chat_model | StrOutputParser()
print(full_chain.invoke("What is a Runnable in LangChain?"))

> **Rules of thumb for `.assign()`**
> - Use it when you already have a dict output and want to **add derived keys**.
> - Great for **progressively enriching** data.
> - The function you pass receives the **entire dict so far** (e.g., `{ 'original_input': ... }`).
> - `.assign()` only **adds** keys; it doesn’t remove or overwrite existing ones.

## ✅ Summary

- **`.bind()`** freezes provider args or generation params on a runnable (great for tool calling and consistent generation settings).
- **`.assign()`** adds new keys to a dict output, perfect for building up prompt variables or enriching intermediate results.
- `RunnableParallel` can be expressed in different syntaxes; **a plain dict in a chain is treated as parallel** by LCEL.

These tools help you write clean, declarative, and reusable chains.